# Introducción a Federated Learning o Aprendizaje Federado

## Problemas de privacidad al usar Deep Learning y otras tecnicas de Machine Learning

Es sabido que las redes neuronales o ```deep learning``` como se lo conoce ahora es un sub-area del campo de ```Machine Learning```. Todo este grupo de algoritmos se caracterisa del resto de las otras áreas de la ```Inteligencia Artificial``` en que hecho de que su principal caracteristica es la capacidad que tienen de aprender utilizando datos, en lugar de usar reglas predefinidas. Pero muchas veces, los datos sobre los que se quiere crear un modelo de machine learning son datos muy personales y privados. Los mejores y más útiles modelos interactúan con los datos más personales de las personas y decirnos cosas sobre nosotros que hubiesen sido dificiles de saber de otra manera. Pero al mismo tiempo dar toda esta información requiere que confiemos en quien va a almacenar estos datos y que los cuidará para protejer nuestra privacidad, lo cual no siempre ocurre. Ejemplos de esto son:
- **Aplicaciones en Medicina**: machine learning puede ayudar a mejorar dramáticamente diagnostico de enfermedades, como detección de tumores en imagenes de MRI, detectar con tiempo retinopatía diabética en imagenes de retina, detección de cancer en imagenes de melanoma, entre varias otras aplicaciónes más. Pero este tipo de datos son bastante sensibles ya que son datos de los pacientes, una filtración de este tipo de información sería muy grave.
- **Recomendaciones**: ya sea recomendacion de productos, contenido o publicidad, estos modelos buscan personalizar la interacción de los usuarios en los servicios que están utilizando. Mientras más información personal del usuario sea posible de obtener para el modelo de recomendación, mucho mejor será la experiencia del usuario final, que recibirá recomendaciones más significativas. En el 2018 se reveló que una empresa de Cambridge utilizó datos personales de varios usuarios de Facebook para crear un perfil psicológico de cada uno y poder crear campañas de desinformación a través de facebook, que recomendaba anunciós con discursos de odio, con  para influenciar campañas electorales en el 2016 en Estados Unidos, influenciar la salida de Inglaterra de la EU (Brexit) entre varios otros escandalos.
    - [Facebook–Cambridge Analytica data scandal](https://www.wikiwand.com/en/Facebook%E2%80%93Cambridge_Analytica_data_scandal)
- **Credit Scoring**: modelos para saber que tan buenos pagadores de prestamos somos. Pueden utilizar informacion personal como historial crediticio, gastos varios y datos demograficos. Esta es información sensible que no querriamos que corra el riesgo de ser revelada a personas mal intencionadas. Por ejemplo, en el 2017 se reveló que Equifax ,una de las más grandes empresas que otorga credit scorings, entre varios otros servicios utilizando información personal de millones de personas, tuvo un breach enorme de información sensible de millones de personas.
    - [Equifax Security Failing](https://www.wikiwand.com/en/Equifax#/Security_failings)
    - [Equifax Breach: What Happened](https://www.csoonline.com/article/3444488/equifax-data-breach-faq-what-happened-who-was-affected-what-was-the-impact.html)
    
Ya que los datos son el recurso primordial para modelos como las redes neuronales, y los casos de uso más significativos de los mismos requiere que interactúen con datos personales, es necesario encontrar una manera de acceder a los mismos sin correr el riesgo de violar la privacidad de las personas.

> Que pasaría si en lugar de acumular datos privados en un lugar centralizado para entrenar un modelo de deep learning, pudieramos enviar el modelo a donde se generan los datos y entrenar el modelo desde ahí, evitando así tener un solo punto de fallo desde el cual pueda ocurrir un ```breach``` de datos.

Esto significa que:
- Tecnicamente para poder participar en el entrenamiento de un modelo de deep learning, los usuarios no necesitan enviar sus datos a nadie, permitiendo así entrenar modelos valiosos con datos de salud, financieros, etc.
- Personas y empresas que antes no podían compartir sus datos por cuestiones legales igual podrán generar valor gracias a ellos.

## Federated Learning

La premisa federated learning es que multiples datasets contienen información que es útil para resolver un problema, pero es dificil poder acceder a estos datasets en cantidades lo suficientemente grandes como para entrenar un modelo de deep learning que generalice lo suficientemente bien.

Si bien el dataset puede tener suficiente informacion para entrenar un modelo de deep learning, la principal preocupación es que este también pueda contener información que no tenga relación con el aprendizaje del modelo, pero que pueda causar daños a alguien si es revelada. 

```Federated Learning``` se trata de enviar el modelo a un entorno seguro y aprender como resolver el problema sin la necesidad de mover los datos a ninguna parte. En este notebook veremos un ejemplo simple de ```federated learning.```

## Referencias
- [Federated Learning: Collaborative Machine Learning without Centralized Training Data](https://ai.googleblog.com/2017/04/federated-learning-collaborative.html)

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
from collections import Counter
import random
import sys
import codecs

np.random.seed(12345)

## Caso de Ejemplo: Detección de SPAM
### Digamos que queremos entrenar un modelo para detectar spam entre los correos de varias personas
Este caso de uso se trata de clasificar correos. Para esto vamos a usar un dataset de correos de ENRON, un dataset publico bastante conocido, por el escandalo generado por dicha empresa. 

### Lectura y preprocesamiento del dataset

In [3]:
vocab, spam, ham = (set(["<unk>"]), list(), list())

# Lecrura de spam
# with codecs.open('datasets/enron-spam/spam.txt', 'r', encoding='utf-8', errors='ignore') as f:
#     raw = f.readlines()
f = codecs.open('datasets/enron-spam/spam.txt', 'r', encoding='utf-8', errors='ignore')
raw = f.readlines()
print('cantidad de mails de spam:', len(raw))
print('Un correo de ejemplo:\n',raw[0])
# test = set(raw[0][:-2].split(" "))
# print(test)
for row in raw:
    # se toma todas las palabras unicas de cada correo
    spam.append(set(row[:-2].split(" ")))
    # por cada una de las palabras del ultimo correo 
    # agregado a la lista de spam
    for word in spam[-1]:
        # se agregan todas las palabras nuevas al vocabulario
        vocab.add(word)

# Repetimos el mismo proceso para el archivo ham.txt
f = codecs.open('datasets/enron-spam/ham.txt', 'r', encoding='utf-8', errors='ignore')
raw = f.readlines()
print('cantidad de mails de ham:', len(raw))
print('Un correo de ejemplo:\n',raw[10])
for row in raw:
    ham.append(set(row[:-2].split(" ")))
    for word in ham[-1]:
        vocab.add(word)

cantidad de mails de spam: 9000
Un correo de ejemplo:
 Subject: dobmeos with hgh my energy level has gone up ! stukm introducing doctor - formulated hgh human growth hormone - also called hgh is referred to in medical science as the master hormone . it is very plentiful when we are young , but near the age of twenty - one our bodies begin to produce less of it . by the time we are forty nearly everyone is deficient in hgh , and at eighty our production has normally diminished at least 90 - 95 % . advantages of hgh : - increased muscle strength - loss in body fat - increased bone density - lower blood pressure - quickens wound healing - reduces cellulite - improved vision - wrinkle disappearance - increased skin thickness texture - increased energy levels - improved sleep and emotional stability - improved memory and mental alertness - increased sexual potency - resistance to common illness - strengthened heart muscle - controlled cholesterol - controlled mood swings - new hair growth a

El codigo anterior es solo preprocesamiento. Lo preprocesamos para tenerlo listo a la hora de hacer forwardprop utilizando ```embeddings```. Algunas caracteristicas importantes del dataset preprocesado para poder entrenar el modelo son:
- Todas las palabras son convertidas en una lista de indices
- Todos los correos son convertidos en listas de 500 palabras exactamente, ya sea recortandolos o rellenandolos con el token ```<unk>```. Hacer esto hace que el dataset sea más fácil de procesar por el modelo

In [4]:
# Tomamos el vocabulario creado y creamos un diccionario
# con las palabras y sus indices
vocab, word2index = (list(vocab), {})
for i, word in enumerate(vocab):
    word2index[word] = i

def to_indices(input, l=500):
    indices = list()
    for line in input:
        # si la linea tiene menos palabras que l
        if (len(line) < l):
            # se completa la linea con el simbolo <unk> tantas
            # veces hasta llegar a una longitud l
            line = list(line) + (['<unk>'] * (l - len(line)))
            idxs = list()
            for word in line:
                idxs.append(word2index[word])
            indices.append(idxs)
    return indices
            

### Creacion de estructuras de datos a ser utilizadas para el entrenamiento de los modelos

In [5]:
# Se optienen los indices de spam y ham
spam_idx = to_indices(spam)
ham_idx = to_indices(ham)

In [6]:
# Agrupamos ham y spam en listas para crear
# los conjuntos de prueba y entrenamiento
train_spam_idx = spam_idx[0:-1000]
train_ham_idx = ham_idx[0:-1000]
test_spam_idx = spam_idx[-1000:]
test_ham_idx = ham_idx[-1000:]

# Creamos los conjuntos de test y entrenamiento
train_data = list()
train_target = list()

test_data = list()
test_target = list()

for i in range(max(len(train_ham_idx), len(train_spam_idx))):
    train_data.append(train_spam_idx[i%len(train_spam_idx)])
    train_target.append([1])
    
    train_data.append(train_ham_idx[i%len(train_ham_idx)])
    train_target.append([0])
    
for i in range(max(len(test_ham_idx), len(test_spam_idx))):
    test_data.append(test_spam_idx[i%len(test_spam_idx)])
    test_target.append([1])
    
    test_data.append(test_ham_idx[i%len(test_ham_idx)])
    test_target.append([0])

### Definicion de las funciones para entrenar y testear el modelo

In [7]:
from lightdlf.cpu.core import Tensor
from lightdlf.cpu.layers import Embedding, MSELoss, CrossEntropyLoss
from lightdlf.cpu.optimizers import SGD
# from lightdlf.cpu.core2 import Tensor, Embedding, MSELoss, SGD

def train(model, input_data, target_data, batch_size=500, iterations=5):
    
    criterion = MSELoss()
    optim = SGD(parameters=model.get_parameters(), alpha=0.01)
    
    n_batches = int(len(input_data) / batch_size)
    for iter in range(iterations):
        iter_loss = 0
        for b_i in range(n_batches):

            # el token auxiliar <unk> se tiene que quedar en 0
            # ya que no debe afectar al modelo
            model.weight.data[word2index['<unk>']] *= 0 
            input = Tensor(input_data[b_i*batch_size:(b_i+1)*batch_size], autograd=True)
            target = Tensor(target_data[b_i*batch_size:(b_i+1)*batch_size], autograd=True)

            pred = model.forward(input).sum(1).sigmoid()
            loss = criterion.forward(pred,target)
            # loss.backward(Tensor(np.ones_like(loss.data)))
            loss.backward()
            optim.step()

            iter_loss += loss.data[0] / batch_size

            sys.stdout.write("\r\tLoss:" + str(iter_loss / (b_i+1)))
        print()
    return model

In [8]:
def test(model, test_input, test_output):
    model.weight.data[word2index['<unk>']] *= 0
    
    input = Tensor(test_input, autograd=True)
    target = Tensor(test_output, autograd=True)
    
    pred = model.forward(input).sum(1).sigmoid()
    return ((pred.data > 0.5) == target.data).mean()

## Entrenamiento de un modelo Centralizado

In [14]:
# model = Embedding(vocab_size=len(vocab), dim=1)
model = Embedding(vocab_size=len(vocab), dim=2)
# model.weight.data *= 0
criterion = MSELoss()
optim = SGD(parameters=model.get_parameters(), alpha=0.01)

In [15]:
for i in range(3):
    model = train(model, train_data, train_target, iterations=1)
    print("% Correct on Test Set: " + str(test(model, test_data, test_target)*100))

	Loss:0.0431399306340297
% Correct on Test Set: 98.275
	Loss:0.012896923366541803
% Correct on Test Set: 99.0
	Loss:0.009016344772140578
% Correct on Test Set: 99.25


In [16]:
model.weight.data[0]

array([0.73603197, 0.81111928])